In [ ]:
%reset

In [ ]:
#### Check if raster and polygon overlap, if true, clip raster using polygon and save it #### 

import ogr, gdal
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs
import os

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

for filename1 in os.listdir('C:\\CUDEM\\'):
    if filename1.endswith('.tif'):
        raster_dir=('C:\\CUDEM\\{0}'.format(filename1))
        raster_name=filename1
        raster_name= raster_name.replace('_2019v2.tif', '')
        raster_name= raster_name.replace('ncei19', '')
        raster = gdal.Open(raster_dir)
        print(raster_dir)
        # Get raster geometry
        transform = raster.GetGeoTransform()
        pixelWidth = transform[1]
        pixelHeight = transform[5]
        cols = raster.RasterXSize
        rows = raster.RasterYSize

        xLeft = transform[0]
        yTop = transform[3]
        xRight = xLeft+cols*pixelWidth
        yBottom = yTop+rows*pixelHeight

        ring = ogr.Geometry(ogr.wkbLinearRing)
        ring.AddPoint(xLeft, yTop)
        ring.AddPoint(xLeft, yBottom)
        ring.AddPoint(xRight, yBottom)
        ring.AddPoint(xRight, yTop)
        ring.AddPoint(xLeft, yTop)
        rasterGeometry = ogr.Geometry(ogr.wkbPolygon)
        rasterGeometry.AddGeometry(ring)
        
        for filename2 in os.listdir('C:\\Users\\Sofia\\Dropbox\\Barriers_US\\NorthCarolina\\Polygons\\'):
            if filename2.endswith('.shp'):
                vector_dir=('C:\\Users\\Sofia\\Dropbox\\Barriers_US\\NorthCarolina\\Polygons\\{0}'.format(filename2))
                vector_name=filename2
                vector = ogr.Open(vector_dir)
                print(vector_dir)
                        
                # Get vector geometry
                layer = vector.GetLayer()
                feature = layer.GetFeature(0)
                vectorGeometry = feature.GetGeometryRef()

                print(rasterGeometry.Intersect(vectorGeometry))
                
                if  rasterGeometry.Intersect(vectorGeometry) == True:
                        # Output raster
                        out_tif = os.path.join('C:\\CUDEM\\CLIP\\CLIP_{0}_{1}.tif'.format(vector_name,raster_name))
                        # Read the data
                        data = rasterio.open(raster_dir)
                        barrier = gpd.read_file(vector_dir)
                        # Project the Polygon into same CRS as the grid
                        barrier = barrier.to_crs(crs=data.crs.data)
                        coords = getFeatures(barrier)
                        # Clip the raster with Polygon
                        out_img, out_transform = mask(dataset=data, shapes=coords, crop=True)
                        # Copy the metadata
                        out_meta = data.meta.copy()
                        # Parse EPSG code
                        epsg_code = int(data.crs.data['init'][5:])
                        out_meta.update({"driver": "GTiff",
                                        "height": out_img.shape[1],
                                        "width": out_img.shape[2],
                                        "transform": out_transform,
                                        "crs": pycrs.parse.from_epsg_code(epsg_code).to_proj4()}
                                                 )
                        with rasterio.open(out_tif, "w", **out_meta) as dest:
                            dest.write(out_img)
                        print(out_tif)
                            
                else:
                    continue

In [ ]:
#### Create mosaic with tiles ####

# Import packages
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline


# File and folder paths
dirpath = "C:\\CUDEM\\CLIP\\"
out_fp = os.path.join(dirpath, "Mosaic_Clip.tif")

# Make a search criteria to select the DEM files
search_criteria = "*.tif"
q = os.path.join(dirpath, search_criteria)
print(q)

# glob function can be used to list files from a directory with specific criteria
dem_fps = glob.glob(q)

# Files that were found:
print(dem_fps)

# List for the source files
src_files_to_mosaic = []

# Iterate over raster files and add them to source -list in 'read mode'
for fp in dem_fps:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

print(src_files_to_mosaic)

# Merge function returns a single mosaic array and the transformation info
mosaic, out_trans = merge(src_files_to_mosaic)

# Copy the metadata
out_meta = src.meta.copy()



# Update the metadata
out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                 "crs": "EPSG:4269"
                 }
                )

# Write the mosaic raster to disk
with rasterio.open(out_fp, "w", **out_meta) as dest:
    dest.write(mosaic)

In [ ]:
#### Extract elevation data for nodes ####
%matplotlib inline
import osmnx as ox
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
from rasterio.plot import show
import numpy as np
# Read polygons
poly = gpd.read_file("C:\\Users\\Sofia\\Dropbox\\Barriers_US\\Polygons\\NC14_geo.shp")
# Extract just the geometry (shapely object) part
poly_geo = poly['geometry'].iloc[0]
# Clean it with a buffer
poly_geo = poly_geo.buffer(0)
poly_geo.is_valid
G = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
# Retrieve nodes and edges
nodes, edges = ox.graph_to_gdfs(G)
# Create an index for the geodataframe nodes
nodes['index'] = range(0, len(nodes))
# Convert to same crs 
nodes = nodes.to_crs("epsg:4269") 

# Open the raster and store metadata
raster= rasterio.open("C:\\CUDEM\\CLIP\\Mosaic_Clip.tif")
    

# Extract point value from raster
values=[]

for point in nodes['geometry']:
    x = point.xy[0][0]
    y = point.xy[1][0]
    row, col=raster.index(x,y)
    print('Point correspond to row, col: %d, %d'%(row,col))
    value=raster.read(1)[row,col]
    print('Raster value on point %.2f \n'%value)
    values.append(value)

Point correspond to row, col: 44479, 40580
Raster value on point 2.46 

Point correspond to row, col: 44485, 40518
Raster value on point 0.90 

Point correspond to row, col: 49228, 34340
Raster value on point 0.69 

Point correspond to row, col: 49213, 34378
Raster value on point 0.64 

Point correspond to row, col: 49212, 34385
Raster value on point 0.67 

Point correspond to row, col: 49197, 34422
Raster value on point 0.90 

Point correspond to row, col: 48308, 36801
Raster value on point 0.64 

Point correspond to row, col: 48301, 36824
Raster value on point 0.55 

Point correspond to row, col: 45844, 40229
Raster value on point 1.39 

Point correspond to row, col: 45045, 40322
Raster value on point 0.82 

Point correspond to row, col: 45039, 40388
Raster value on point 1.10 

Point correspond to row, col: 47903, 38768
Raster value on point 2.14 

Point correspond to row, col: 47785, 38742
Raster value on point 2.82 

Point correspond to row, col: 37164, 41697
Raster value on point